# <center>CZ2001 Algorithms Assignment 1<center>

This notebook will go through the ***Brute Force Algorithm*** and the ***Knuth-Morris-Pratt (KMP) Pattern Searching Algorithm*** for Substring Search. It will be done in <font color = green>Python 3.6.9</font>.

For this notebook (and our report), we will be comparing the substring searching algorithm(s) with a .fna file containing the genome sequence of Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) reference genome [found here](https://www.ncbi.nlm.nih.gov/assembly/?term=Severe+acute+respiratory+syndrome+coronavirus+2).

Users should upload their own .fna file to test the algorithm. Should the first cell of the notebook (used for uploading files) hang, please enter the current session's directory located on the navigation bar to the left, click on the folder, and drag and drop the .fna file into the directory.
<br>
<br>
<br>
Lab Group SSP2, Done by:
- Low Yu Benedict
- Leong Kah Wai Alex
- Ong Jing Hong Elliott
- Sunny Pek Yee Chong
- Loe Kit Leong Daniel

## <center> Constraints

The main constraint of using the Google Colab environment is that it is not optimized for performance computing and has factors like latency and Google's dynamic allocation of computing resource to deal with.

As such, this notebook will present findings with graphs that are slightly erratic due to the nature of how computations performed.

## <center>Usage
This notebook is written for the Google Colab working environment. Users can access the Google Colab notebook [using this link](https://colab.research.google.com/drive/1KCj2Ei0D9H3_MxnQNgDoBqj80XtmSJxl?usp=sharing). The maximum upload file size for Google Colab is 80GB, and 350GB if a GPU accelerator is used. However, due to the limited computing resource and bandwidth on Google Colab, uploading a file size of even 300MB take more than 2 hours.

As such, <font color = red>***it is recommended that files of no larger than 5MB be uploaded***</font> in this Google Colab environment for a smooth experience.

Below are download links to some organisms that fulfill this criteria:
- [Amaranthus tuberculatus](https://www.ncbi.nlm.nih.gov/assembly/GCA_000180655.1/) (5MB~)
- [Severe acute respiratory syndrome coronavirus 2](https://www.ncbi.nlm.nih.gov/assembly/?term=Severe+acute+respiratory+syndrome+coronavirus+2) (30KB~, Recommended for a smooth experience on this notebook)

There are two ways to upload your own file:

1. To <font color = red>***upload a file, you can run the first cell***</font> in this notebook then upload your file. ***Do not run the second cell of the notebook for this option.*** Sometimes there may be a bug (on Google Colab's conflict with browser) "TypeError: Cannot read property '_uploadFiles' of undefined". If this happens you will have to resort to option 2.

2. Or, for more proficient users you may simply drag and drop the .fna file into the directory on the left navigation bar (navigation bar -> file icon -> drag and drop). If you <font color = red>***dragged and dropped the .fna file, then run from the second cell only***.</font> ***Do not run the first cell of the notebook for this option.***

## <center>Report


This link leads to a [detailed 5-page](https://docs.google.com/document/d/1LQtYf26k0NhYeA8KY6r7qGrkDNYueBonc6yxCRQ0yfo/edit?usp=sharing) report on Google Docs.

In [2]:
# Run this cell to upload file
from google.colab import files
uploaded = files.upload()
fna_directory, _ = list(uploaded.items())[0]

Saving GCF_009858895.2_ASM985889v3_genomic.fna to GCF_009858895.2_ASM985889v3_genomic.fna


In [ ]:
# UNCOMMENT AND RUN THIS CELL IF YOU DRAGGED AND DROPPED FILE DIRECTLY

# import os

# # print list of files in directory
# for i in range (len(os.listdir())):
#   print(str(i)+") " + os.listdir()[i])

# # take in user's input for which file they want
# chosen_file = input("Select the index where the name of the file you uploaded is at (indices start from 0):\n ")
# fna_directory = os.listdir()[int(chosen_file)] 

# print("You have selected the file: " + fna_directory)

0) .config
1) GCF_009858895.2_ASM985889v3_genomic.fna
2) sample_data
Select the index where the name of the file you uploaded is at (indices start from 0):
 2
You have selected the file: sample_data


In [3]:
!pip install biopython

     |████████████████████████████████| 2.3MB 8.7MB/s 


In [37]:
from Bio import SeqIO

sequences = []  # Setup an empty list
for record in SeqIO.parse(fna_directory, "fasta"):
  sequences.append(record)

print("Total of " + str(len(sequences)) + " sequence(s) recorded.")

for i in range(len(sequences)):
  print(str(i)+") " + " ID = " + sequences[i].id + ", Name = " + sequences[i].name +", Genome Sequence Length (Number of Characters): " + str(len(sequences[i].seq))) 

# Error will occur if user enters an input out of bounds.
user_seq = input("\nWhich sequence would you like to choose?\n")
fna_string = str(sequences[int(user_seq)].seq)
print("\nSequence selected. First 1000 characters of selected genome sequences:\n")
fna_string[:1000]

Total of 1 sequence(s) recorded.
0)  ID = NC_045512.2, Name = NC_045512.2, Genome Sequence Length (Number of Characters): 29903

Which sequence would you like to choose?
0

Sequence selected. First 1000 characters of selected genome sequences:



'ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTCGTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCTTCTTCGTAAGAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTAGGCGACGAGCTTGGCACTGATCCTTATGAAGATTTTCAAGAAAACTGGAACACTAAACATAGCAGTGGTGTTACCCGTGAACTCATGCGTGAGCTTAACGGAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGGCCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGGTAAAGCTTCATGCACTTTGTCCGAACAACTGGACTTTATTGACACTAAGAGGGGTGTATACTGCTGCCGTGAACATGAGCATGAAATTGCTTGGTACACGGAACGTTC

In [5]:
len(fna_string)

29903

In [6]:
# Run this cell to enter substring
fna_substring = input("Enter Substring: ")

Enter Substring: ATTA


## <center>Brute Force Algorithm Search

### Explanation

1. Traverses the main string from start (left) to end (right)

2. Check every character from the main string to match against the first character of the substring

3. If the first character matches, keep matching while iterating through both the main string and substring till the last character of the substring. If there is a mismatch at any point, continue the traversal of the main string after where the first character matched previously and repeat from step 2

4. If there is no mismatch between the main string and substring throughout the subsequent iterations after the first character matches, substring is found




In [7]:
def brute_force(substring, string):
  '''
  This function performs the Brute Force Search algorithm search.
  
  Parameters
  ----------
  string : str
      String to be iterated through to find the occurences of a substring.
  substring: str 
      String to be searched for.

  Returns
  -------
  list
      A list containing the positions of each substring occurence. An empty
      list is returned if no substring can be found.
  '''

  substring_index = []
  string_length = len(string)
  substring_length = len(substring)

  # Traverse the string
  for i in range (string_length):
    # Set our flag to True 
    substring_occurrence_complete = True
    # match found at start of substring                  
    if substring[0] == string[i]:
      # confirm that there is enough character down the string to contain a 
      # substring. This is to avoid premature confirmation of substrings with
      # only a partial match. 
        # Traverse the substring
        for j in range (0, substring_length):
          # If there is a mismatch, break and return to the main for loop
          if i+1 >= string_length:
            substring_occurrence_complete = False
          elif substring[j] != string[i+j]:
            substring_occurrence_complete = False
            break
        # Match has been found, append to list
        if substring_occurrence_complete:
          substring_index.append(i+1)
    
  return substring_index

In [8]:
# Uncomment the line below and enter your own substring for ease of testing.
# fna_substring = input("Enter Substring: ")

bf_results = brute_force(fna_substring, fna_string)
print("Substring found at positions: " + str(bf_results)) if bf_results else print("No substring found.")

Substring found at positions: [1, 132, 143, 866, 1034, 1688, 1749, 2066, 2276, 2345, 2480, 2560, 2618, 2929, 3126, 3298, 3383, 3531, 3706, 3709, 4040, 4236, 4340, 4426, 4490, 4515, 4872, 4967, 4994, 5212, 5264, 5361, 5754, 5852, 5958, 5961, 6177, 6488, 6521, 6584, 6601, 6613, 6684, 6765, 6833, 6912, 6950, 6970, 7190, 7340, 7370, 7437, 7658, 7877, 7990, 8360, 8585, 8884, 9146, 9291, 9294, 9741, 9859, 10212, 10460, 10512, 10552, 10652, 10767, 10855, 10864, 10912, 11117, 11252, 11430, 11700, 11789, 12407, 12605, 12679, 12956, 12964, 13063, 13110, 13863, 13904, 13953, 14058, 14150, 14172, 14236, 14550, 14601, 14748, 14795, 14902, 14984, 15082, 15162, 15293, 15334, 15402, 15800, 15914, 16275, 16356, 16445, 16471, 16592, 16722, 16772, 16890, 16904, 16938, 16981, 17039, 17199, 17233, 17289, 17378, 17406, 17448, 17698, 17929, 18227, 18740, 18952, 18999, 19002, 19051, 19134, 19362, 19376, 19430, 19699, 19764, 19807, 20047, 20118, 20150, 20153, 20184, 20265, 20298, 20412, 20511, 20652, 20721, 20

## <center>Knuth-Morris-Pratt (KMP) Pattern Searching Algorithm

### Explanation

1. (Pre-Processing) Build an array containing information of the longest common prefix and suffix which do not overlap in the substring.

2. Traverses the main string and the substring from start (left) to end (right) while at each traversal step, matching the characters from both main and substring.

3. If there is a mismatch of character at any point before the traversal completes iterating through the complete substring, a secondary step has to be performed on the substring where the largest common prefix and suffix before the mismatched character have to be identified using the constructed array.

4. Once the common prefix and suffix have been identified in the substring, the traversal of the main string will not move to the next character but remain at its current index and try to match the mismatched character again but this time with the character after the prefix in the substring. If both characters do not match, the main string’s search index will be moved to the next index while traversal of the substring restarts at its first character to begin matching again.

5. If the previously mismatched character in the main string and character after the prefix in substring matches, traversal and matching for both the main string and substring will continue either till the successful matching of the last character of the substring which indicates that the substring has been found or until there is a mismatch once again where the process flow will repeat from step 2.



In [9]:
def KMP_Search(substring, string):
  '''
  This function performs the Knuth-Morris-Pratt Pattern Searching Algorithm WITH
  the Pre-Processing stage.
  
  Parameters
  ----------
  string : str
      String to be iterated through to find the occurences of a substring.
  substring: str 
      String to be searched for.

  Returns
  -------
  list
      A list containing the positions of each substring occurence. An empty
      list is returned if no substring can be found.
  '''

  # Initialize Variables
  substring_len = len(substring) 
  string_len = len(string)
  substring_position = []
  j = 0 
  i = 0 
  # Longest Prefix that is also a Proper Suffix
  lpps = KMP_Preprocess(substring, substring_len)

  # Traverse the string length
  while i < string_len: 

    # If a match is found, increment i (string)
    # and j (substring) indices.
    if substring[j] == string[i]: 
      i += 1
      j += 1

    # If a complete match is found, append position to list. Readjust j
    # to the previous element in the lpps array as that is where the last 
    #occurence of an lpps was recorded . That would mean you should now begin 
    # searching from that index. For example,with substring dsgwdsg, you will 
    # now go back to search "w" onwards.
    if j == substring_len: 
      substring_position.append(i-j+1) 
      j = lpps[j-1] 

    # While ensuring we are within index, check if there's a 
    # mismatch
    elif i < string_len and substring[j] != string[i]:
      # If j!=0 we know that there is still a valid non-zero lpps entry in the 
      # list. Therefore, traverse the lpps list to discover if there are 
      # previous possible lpps to continue from (go back to while loop and 
      # try again), do not increment i
      if j != 0: 
        j = lpps[j-1] 
      # Otherwise we know that there's no possible lpps left, and
      # we just increment i while j = 0
      else: 
        i += 1
  return substring_position, lpps

def KMP_Preprocess(substring, substring_len): 
  '''
  This function performs the Knuth-Morris-Pratt Search Pre-Processing.
  
  Parameters
  ----------
  substring: str 
      String to be pre-processed.
  substring_len: int
      The length of the substring.

  Returns
  -------
  list
      A list of indices for each indice's longest prefix that is also a proper 
      suffix.
  '''

  # Initialize Variables
  # latest_lpps_idx is used for keeping track of the most recent index
  # where an lpps was recorded
  latest_lpps_idx = 0
  # Longest Prefix that is also a Proper Suffix
  lpps = [0]*substring_len 
  i = 1

  # Traverse the substring
  while i < substring_len:
    # Matching character, update lpps list with the latest_lpps_idx + 1
    if substring[i] == substring[latest_lpps_idx]: 
      latest_lpps_idx += 1
      lpps[i] = latest_lpps_idx
      i += 1
    else: 
      # No matching character, traverse backwards of the lpps list to see
      # if we can pick up from any spot where we left off. The idea is that
      # we are trying to find if perhaps we can continue from an lpps we
      # left off from.
      if latest_lpps_idx != 0: 
        latest_lpps_idx = lpps[latest_lpps_idx-1] 
      # Append 0 as this means we couldn't find an lpps to continue from
      else: 
        lpps[i] = 0
        i += 1
  return lpps

In [10]:
def Separate_PrepKMP(substring, string, lpps):
  '''
  This function performs the Knuth-Morris-Pratt Pattern Searching Algorithm
  WITHOUT the Pre-Processing Stage. It takes the pre-processed list as an
  input.
  
  Parameters
  ----------
  string : str
      String to be iterated through to find the occurences of a substring.
  substring: str 
      String to be searched for.
  lpps: list
      A list performed during the pre-process stage. It contains the list
      of longest-prefix-that-is-also-a-proper-suffix.

  Returns
  -------
  list
      A list containing the positions of each substring occurence. An empty
      list is returned if no substring can be found.
  '''

  # Initialize Variables
  substring_len = len(substring) 
  string_len = len(string)
  substring_position = []
  j = 0 
  i = 0 

  # Traverse the string length
  while i < string_len: 

    # If a match is found, increment i (string)
    # and j (substring) indices.
    if substring[j] == string[i]: 
      i += 1
      j += 1

    # If a complete match is found, append position to list. Readjust j
    # to the previous element in the lpps array as that is where the last 
    #occurence of an lpps was recorded . That would mean you should now begin 
    # searching from that index. For example,with substring dsgwdsg, you will 
    # now go back to search "w" onwards.
    if j == substring_len: 
      substring_position.append(i-j+1) 
      j = lpps[j-1] 

    # While ensuring we are within index, check if there's a 
    # mismatch
    elif i < string_len and substring[j] != string[i]:
      # If j!=0 we know that there is still a valid non-zero lpps entry in the 
      # list. Therefore, traverse the lpps list to discover if there are 
      # previous possible lpps to continue from (go back to while loop and 
      # try again), do not increment i
      if j != 0: 
        j = lpps[j-1] 
      # Otherwise we know that there's no possible lpps left, and
      # we just increment i while j = 0
      else: 
        i += 1
  return substring_position, lpps

In [11]:
# Uncomment the line below and enter your own substring for ease of testing.
# fna_substring = input("Enter Substring: ")

kmp_results, lpps = KMP_Search(fna_substring, fna_string)
print("The pre-processing array (longest prefix that is also a suffix) is: " + str(lpps))
print("Substring found at positions: " + str(kmp_results)) if kmp_results else print("No substring found.")

The pre-processing array (longest prefix that is also a suffix) is: [0, 0, 0, 1]
Substring found at positions: [1, 132, 143, 866, 1034, 1688, 1749, 2066, 2276, 2345, 2480, 2560, 2618, 2929, 3126, 3298, 3383, 3531, 3706, 3709, 4040, 4236, 4340, 4426, 4490, 4515, 4872, 4967, 4994, 5212, 5264, 5361, 5754, 5852, 5958, 5961, 6177, 6488, 6521, 6584, 6601, 6613, 6684, 6765, 6833, 6912, 6950, 6970, 7190, 7340, 7370, 7437, 7658, 7877, 7990, 8360, 8585, 8884, 9146, 9291, 9294, 9741, 9859, 10212, 10460, 10512, 10552, 10652, 10767, 10855, 10864, 10912, 11117, 11252, 11430, 11700, 11789, 12407, 12605, 12679, 12956, 12964, 13063, 13110, 13863, 13904, 13953, 14058, 14150, 14172, 14236, 14550, 14601, 14748, 14795, 14902, 14984, 15082, 15162, 15293, 15334, 15402, 15800, 15914, 16275, 16356, 16445, 16471, 16592, 16722, 16772, 16890, 16904, 16938, 16981, 17039, 17199, 17233, 17289, 17378, 17406, 17448, 17698, 17929, 18227, 18740, 18952, 18999, 19002, 19051, 19134, 19362, 19376, 19430, 19699, 19764, 19807

## <center> Testing and Comparing Search Results

As sets take the unique, and similar pairs of values, if the size of the set <font color=green>**"comparison"**</font> matches the size of both search algorithm's returned list, that means that both algorithms matched the substring occurences at the same positions.

In [12]:
comparison = set(bf_results) & set (kmp_results)
print("Size of resulting set = " + str(len(comparison)) + ", \nSize of bf_results = " + str(len(bf_results)) + ", \nSize of kmp_results = " + str(len(kmp_results)))

Size of resulting set = 219, 
Size of bf_results = 219, 
Size of kmp_results = 219


For ease of testing and usability, you may run the cells below to try different substrings vs the return results of the Brute Force and KMP Search Algorithm.

In [22]:
print(fna_string[:1000]) #print first 1000 characters to reference
string = fna_string #The genome sequences uploaded earlier
substring = input("Enter a substring: ")

bf_ = brute_force(substring, string)
kmp_, _ = KMP_Search(substring, string)

comparison = set(bf_) & set(kmp_)

print("\nThe pre-processing array (longest prefix that is also a suffix) is: " + str(lpps))
print("(KMP) Substring found at positions        : " + str(kmp_results)) if kmp_results else print("No substring found.")
print()
print("(Brute Force) Substring found at positions: " + str(bf_results)) if bf_results else print("No substring found.")
print()
print("Size of resulting set = " + str(len(comparison)) + ", \nSize of bf_results = " + str(len(bf_)) + ", \nSize of kmp_results = " + str(len(kmp_)))

ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTCGTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCTTCTTCGTAAGAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTAGGCGACGAGCTTGGCACTGATCCTTATGAAGATTTTCAAGAAAACTGGAACACTAAACATAGCAGTGGTGTTACCCGTGAACTCATGCGTGAGCTTAACGGAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGGCCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGGTAAAGCTTCATGCACTTTGTCCGAACAACTGGACTTTATTGACACTAAGAGGGGTGTATACTGCTGCCGTGAACATGAGCATGAAATTGCTTGGTACACGGAACGTTCT

## <center> Comparison between Brute Force Algorithm and KMP Pattern Search Algorithm

In [27]:
# list of 20 substrings that exists in the genome sequence.
existing_ss_list = [fna_string[:i] for i in range(2, 2000, 10)]

# list of 20 substrings that does not exist in the genome sequence.
nonexistent_ss_list = [fna_string[:i] + '-' for i in range(2, 2000, 10)]

substring_length = [len(fna_string[:i] + '-') for i in range(2, 2000, 10)]

In [28]:
len(substring_length)

200

## <center> Comparing KMP with Brute Force (Non-Existent Substring)

In [29]:
def replace_anomaly_with_avg(item):
  '''
  This function replaces anomalies in the dataset with an average value of the
  adjacent points. This is only done for the user's viewing experience as the 
  anomalies tends to occur only once in each of the algorithm's 200 data point
  resulting in a plotted graph looking very out of proportion.

  Parameters
  ----------
  list : item
      A list containing a list of the algorithm's recorded run time.

  Returns
  -------
  list
      A list containing an adjusted list of the algorithm's recorded run time.
  '''
  item_length = len(item)

  for i in range (1, item_length-1):
    if item[i] > (item[i-1]+item[i+1])/2 + 0.25:
      print("Anomaly found, value at: " + str(item[i]) + ", while neighbouring values are " + str(item[i-1]) + "," + str(item[i+1]))
      item[i] = round((item[i-1]+item[i+1])/2, 2)

  return item

In [30]:
# Do note this will take some time to run, as we are running many iterations of
# each searching algorithm and logging the time taken. 100 iterations of each
# data point and we record the average of the best 3 iteration for each point. 
# In other words, we are running 100 * 200 = 20,000 iterations for 
# each searching algorithm (total of 60, 000 iterations for 3 algorithms).

import time

nonexistent_bf_logs = []
nonexistent_kmp_logs = []
nonexistent_noprep_kmp_logs = []

iter = len(substring_length)

for i in range (iter): 
  lpps = KMP_Preprocess(nonexistent_ss_list[i], len(nonexistent_ss_list[i]))
  timing = %timeit -o Separate_PrepKMP(nonexistent_ss_list[i], fna_string, lpps)
  nonexistent_noprep_kmp_logs.append(round(timing.best*1000, 2))
  if i%20 == 0:
    print("Completed KMP Omit Prep Iteration " + str(i) + ". Time Taken: " + str(nonexistent_noprep_kmp_logs[-1]))

print()

for i in range (iter): 
  timing = %timeit -o KMP_Search(nonexistent_ss_list[i], fna_string)
  nonexistent_kmp_logs.append(round(timing.best*1000, 2))
  if i%20 == 0:
    print("Completed KMP Iteration " + str(i) + ". Time Taken: " + str(nonexistent_kmp_logs[-1]))

print()

for i in range (iter): 
  timing = %timeit -o brute_force(nonexistent_ss_list[i], fna_string)
  nonexistent_bf_logs.append(round(timing.best*1000, 2))
  if i%20 == 0:
    print("Completed Brute Force Iteration " + str(i) + ". Time Taken: " + str(nonexistent_bf_logs[-1]))




100 loops, best of 3: 7.21 ms per loop
Completed KMP Omit Prep Iteration 0. Time Taken: 7.21
100 loops, best of 3: 7.2 ms per loop
100 loops, best of 3: 7.23 ms per loop
100 loops, best of 3: 7.23 ms per loop
100 loops, best of 3: 7.22 ms per loop
100 loops, best of 3: 7.24 ms per loop
100 loops, best of 3: 7.28 ms per loop
100 loops, best of 3: 7.37 ms per loop
100 loops, best of 3: 7.24 ms per loop
100 loops, best of 3: 7.28 ms per loop
100 loops, best of 3: 7.21 ms per loop
100 loops, best of 3: 7.28 ms per loop
100 loops, best of 3: 7.21 ms per loop
100 loops, best of 3: 7.23 ms per loop
100 loops, best of 3: 7.21 ms per loop
100 loops, best of 3: 7.22 ms per loop
100 loops, best of 3: 7.29 ms per loop
100 loops, best of 3: 7.23 ms per loop
100 loops, best of 3: 7.22 ms per loop
100 loops, best of 3: 7.24 ms per loop
100 loops, best of 3: 7.24 ms per loop
Completed KMP Omit Prep Iteration 20. Time Taken: 7.24
100 loops, best of 3: 7.2 ms per loop
100 loops, best of 3: 7.22 ms per l

In [31]:
# Adjust for anomalies
# As the graph can be erratic at some occassional data point, as a result of
# bad bandwidth or Google Colab dynamically allocating a reduction of 
# computational resource, we will adjust anomalies to be the average of the
# two adjacent data points.

nonexistent_bf_logs = replace_anomaly_with_avg(nonexistent_bf_logs)
nonexistent_kmp_logs = replace_anomaly_with_avg(nonexistent_kmp_logs)
nonexistent_noprep_kmp_logs = replace_anomaly_with_avg(nonexistent_noprep_kmp_logs)

Anomaly found, value at: 15.68, while neighbouring values are 8.39,8.36


In [32]:
import plotly.express as px

fig = px.scatter()
fig.update_layout(
    title="KMP Search vs Brute Force for Nonexistent Substrings",
    xaxis_title = "Length of Substring",
    yaxis_title = "Runtime (ms)"
    )

fig.add_scatter(x=substring_length, y=nonexistent_bf_logs,name="Brute Force")
fig.add_scatter(x=substring_length, y=nonexistent_kmp_logs, name="KMP with Prep Time")
fig.add_scatter(x=substring_length, y=nonexistent_noprep_kmp_logs, name="KMP w/o Prep Time")

fig.show()

## <center> Comparing KMP with Brute Force (Existing Substring)

In [33]:
# Do note this will take some time to run, as we are running many iterations of
# each searching algorithm and logging the time taken. 100 iterations of each
# data point and we record the average of the best 3 iteration for each point. 
# In other words, we are running 100 * 200 = 20,000 iterations for 
# each searching algorithm (total of 60, 000 iterations for 3 algorithms).

import time

existent_bf_logs = []
existent_kmp_logs = []
existent_noprep_kmp_logs = []

iter = len(substring_length)

for i in range (iter): 
  lpps = KMP_Preprocess(existing_ss_list[i], len(existing_ss_list[i]))
  timing = %timeit -o Separate_PrepKMP(existing_ss_list[i], fna_string, lpps)
  existent_noprep_kmp_logs.append(round(timing.best*1000, 2))
  if i%20 == 0:
    print("Completed KMP Omit Prep Iteration " + str(i) + ". Time Taken: " + str(existent_noprep_kmp_logs[-1]))

print()

for i in range (iter): 
  timing = %timeit -o KMP_Search(existing_ss_list[i], fna_string)
  existent_kmp_logs.append(round(timing.best*1000, 2))
  if i%20 == 0:
    print("Completed KMP Iteration " + str(i) + ". Time Taken: " + str(existent_kmp_logs[-1]))

print()

for i in range (iter): 
  timing = %timeit -o brute_force(existing_ss_list[i], fna_string)
  existent_bf_logs.append(round(timing.best*1000, 2))
  if i%20 == 0:
    print("Completed Brute Force Iteration " + str(i) + ". Time Taken: " + str(existent_bf_logs[-1]))




100 loops, best of 3: 7.32 ms per loop
Completed KMP Omit Prep Iteration 0. Time Taken: 7.32
100 loops, best of 3: 7.21 ms per loop
100 loops, best of 3: 7.27 ms per loop
100 loops, best of 3: 7.22 ms per loop
100 loops, best of 3: 7.19 ms per loop
100 loops, best of 3: 7.25 ms per loop
100 loops, best of 3: 7.2 ms per loop
100 loops, best of 3: 7.2 ms per loop
100 loops, best of 3: 7.28 ms per loop
100 loops, best of 3: 7.23 ms per loop
100 loops, best of 3: 7.29 ms per loop
100 loops, best of 3: 7.27 ms per loop
100 loops, best of 3: 7.29 ms per loop
100 loops, best of 3: 7.21 ms per loop
100 loops, best of 3: 7.26 ms per loop
100 loops, best of 3: 7.22 ms per loop
100 loops, best of 3: 7.28 ms per loop
100 loops, best of 3: 7.22 ms per loop
100 loops, best of 3: 7.28 ms per loop
100 loops, best of 3: 7.27 ms per loop
100 loops, best of 3: 7.24 ms per loop
Completed KMP Omit Prep Iteration 20. Time Taken: 7.24
100 loops, best of 3: 7.26 ms per loop
100 loops, best of 3: 7.22 ms per l

In [34]:
# Adjust for anomalies

existent_bf_logs = replace_anomaly_with_avg(existent_bf_logs)
existent_kmp_logs = replace_anomaly_with_avg(existent_kmp_logs)
existent_noprep_kmp_logs = replace_anomaly_with_avg(existent_noprep_kmp_logs)

Anomaly found, value at: 7.9, while neighbouring values are 7.52,7.52
Anomaly found, value at: 14.56, while neighbouring values are 7.64,7.61
Anomaly found, value at: 14.19, while neighbouring values are 7.17,7.25


In [35]:
import plotly.express as px


fig = px.scatter()
fig.update_layout(
    title="KMP Search vs Brute Force for Existing Substrings",
    xaxis_title = "Length of Substring",
    yaxis_title = "Runtime (ms)"
    )

fig.add_scatter(x=substring_length, y=existent_bf_logs,name="Brute Force")
fig.add_scatter(x=substring_length, y=existent_kmp_logs, name="KMP with Prep Time")
fig.add_scatter(x=substring_length, y=existent_noprep_kmp_logs, name="KMP w/o Prep Time")

fig.show()